In [1]:
import numpy as np
import pandas as pd
import pyBigWig
import itertools
sp_file='http://smithdata.usc.edu/methbase/data/Wang-Mouse-2014/Mouse_Sperm/tracks_mm10/Mouse_Sperm.hmr.bb'
ftp='ftp://ftp.ensembl.org/pub/release-102/fasta/mus_musculus/dna/'

def get_seq(chrs):
    asmbl={}
    for ch in chrs:
        print(ch,end =",")
        asmbl[ch]=ftp+'Mus_musculus.GRCm38.dna_rm.chromosome.'+str(ch)+'.fa.gz'
        asmbl[ch]=pd.read_csv(asmbl[ch])
        asmbl[ch]=''.join(asmbl[ch].iloc[:,0].tolist())
    return (asmbl)    

def ranges(i):
    for a, b in itertools.groupby(enumerate(i), lambda pair: pair[1] - pair[0]):
        b = list(b)
        yield b[0][1], b[-1][1]
        

In [2]:
bb=pyBigWig.open(sp_file)
sp=bb.entries('chr19', 0, -1)
len(sp)
sps=[]
for i in sp:
    sps.append(set(range(i[0],i[1])))
sps=frozenset().union(*sps)
print(len(sps))
np.max(list(sps))

3405887


61331127

In [3]:
cgi=pd.read_table('cm19',index_col=0)
print(len(cgi))

534


In [7]:
cgi['i']=cgi.apply(lambda x:(x['chromStart'],x['chromEnd']),1)
cgis=[]
for i in cgi.i.values:
    cgis.append(set(range(i[0],i[1])))
cgis=frozenset().union(*cgis)
print(len(cgis))

360833


In [8]:
len(cgis&sps)

356882

In [9]:
len(cgis&sps)/len(cgis)

0.9890503363051606

In [10]:
asmbl=get_seq([19])
len(asmbl[19])

19,

61431566

In [19]:
ch=19
d=1000
cgms=[]
for b in range(len(asmbl[ch])-d):
    c=asmbl[ch][b-d:b+d].count('C')
    g=asmbl[ch][b-d:b+d].count('G')
    if c+g>d:
        cg=asmbl[ch][b-d:b+d].count('CG')
        if cg>0:
            tg=asmbl[ch][b-d:b+d].count('TG')
            ca=asmbl[ch][b-d:b+d].count('CA')
            if tg+ca==0:
                cgms.append(b) 
            else:
                m=(2*cg)/(tg+ca)
                if m>.5:
                   cgms.append(b) 

In [20]:
cgmsl=list(ranges(list(cgms)))

In [21]:
cgmsl=[c for c in cgmsl if c[1]-c[0]>200]

In [22]:
len(cgmsl)

340

In [23]:
cgmss=[]
for i in cgmsl:
    cgmss.append(set(range(i[0],i[1])))
cgmss=frozenset().union(*cgmss)
len(cgmss)

439495

In [24]:
len(cgmss&sps)

431998

In [25]:
len(cgmss&sps)/len(cgmss)

0.9829417854583101